In [18]:
from importlib import reload

In [4468]:
import medvqa
from medvqa.metrics.medical.med_completeness import (
    MedicalCompleteness,
    WeightedMedicalCompleteness,
    MEDICAL_TERMS_PATH, 
    MEDICAL_SYNONYMS_PATH,
)
from medvqa.datasets.tokenizer import Tokenizer
from nltk.tokenize import wordpunct_tokenize
import math

In [4467]:
reload(medvqa.metrics.medical.med_completeness)
reload(medvqa.datasets.tokenizer)

<module 'medvqa.datasets.tokenizer' from '/home/pamessina/medvqa/medvqa/datasets/tokenizer.py'>

In [132]:
_gt_s = 'there is no free air under the hemidiaphragms . low lung volumes but no acute process and no evidence of free peritoneal air .'
_gt_ids = tokenizer.string2ids(_gt_s)
_gen_s = 'no free air below the right hemidiaphragm is seen .'
_gen_ids = tokenizer.string2ids(_gen_s)
medcomp.score__debug(_gt_ids, _gen_ids, tokenizer, verbose=False)

------------------
ground truth
-----
1-grams (16): no, free, air, below, hemidiaphragm, low, lung, volume, no, acute, process, no, evidence, free, peritoneal, air
ignored (10): <s>, there, is, the, ., but, and, of, ., </s>
------------------
generated
-----
1-grams (6): no, free, air, below, right, hemidiaphragm
ignored (6): <s>, the, is, seen, ., </s>
intersec_size = [5, 3, 2, 1]


0.2754419191919192

In [133]:
from medvqa.datasets.mimiccxr import MIMICCXR_QA_ADAPTED_REPORTS_JSON_PATH
from medvqa.datasets.iuxray import IUXRAY_QA_ADAPTED_REPORTS_JSON_PATH
from medvqa.utils.files import load_json_file
from medvqa.datasets.preprocessing import get_sentences
import random

In [102]:
iuxray_qa_reports = load_json_file(IUXRAY_QA_ADAPTED_REPORTS_JSON_PATH)
mimiccxr_qa_reports = load_json_file(MIMICCXR_QA_ADAPTED_REPORTS_JSON_PATH)

In [119]:
sentences = [None] * (10 * (len(iuxray_qa_reports['reports']) + len(mimiccxr_qa_reports['reports'])))
for i, s in enumerate(get_sentences([iuxray_qa_reports, mimiccxr_qa_reports])):
    sentences[i] = s
sentences = sentences[:i]

In [136]:
tokenized_sentences = [wordpunct_tokenize(s) for s in sentences]

In [160]:
word2idxs = dict()
for i, s in enumerate(tokenized_sentences):
    for w in s:
        try:
            word2idxs[w].add(i)
        except KeyError:
            word2idxs[w] = {i}
for w in word2idxs.keys():
    word2idxs[w] = list(word2idxs[w])

In [4264]:
len(word2idxs)

11214

In [4138]:
def clean_medical_terms_file():
    with open('./medical_terms__aux.txt') as in_f:
        medical_terms = [x.strip() for x in in_f.readlines()]
        medical_terms = sorted(list(set(medical_terms)))
        for x in medical_terms:
            assert x in word2idxs, x
    with open('./medical_terms.txt', 'w') as out_f:
        for x in medical_terms:
            out_f.write(x + '\n')

def clean_medical_synonyms_file():
    with open('./medical_synonyms__aux.txt') as in_f:
        medical_synonyms = [x.strip().split() for x in in_f.readlines()]
        terms = set()
        for x in medical_synonyms:            
            for y in x:
                assert y in word2idxs, y
                size_bef = len(terms)
                terms.add(y)
                try:
                    assert len(terms) == size_bef + 1
                except AssertionError:
                    print(y)
                    raise
        for i in range(len(medical_synonyms)):
            medical_synonyms[i].sort()
        medical_synonyms.sort(key=lambda x:x[0])
    with open('./medical_synonyms.txt', 'w') as out_f:
        for x in medical_synonyms:
            out_f.write(f'{" ".join(x)}\n')

In [4481]:
clean_medical_terms_file()
clean_medical_synonyms_file()

In [4287]:
tokenizer = Tokenizer('vocab/test.pkl', [mimiccxr_qa_reports, iuxray_qa_reports], min_freq=5, overwrite=True)

1150867it [00:06, 167813.31it/s]


In [4289]:
len(tokenizer.id2token)

5013

In [4261]:
tokenizer.id2token[:20]

['<pad>',
 '<s>',
 '</s>',
 '"',
 "'",
 ',',
 '.',
 ':',
 '>',
 'CHF',
 'COPD',
 'a',
 'aa',
 'aaa',
 'abandoned',
 'abdomen',
 'abdominal',
 'aberrant',
 'ability',
 'ablation']

In [4482]:
medcomp = MedicalCompleteness(tokenizer, device='cpu')

In [4487]:
_word = 'third'
len(word2idxs[_word]), sentences[random.choice(word2idxs[_word])]

(650, 'possible subtle fracture along anterolateral right third rib.')

In [4164]:
def rank_sentence(s):
    return sum(1 for token in s if token in medcomp.medical_terms)
    
def sample_sentences(n_samples=1000, top_k=5):
    idxs = random.sample(range(len(sentences)), n_samples)
    idxs.sort(key=lambda i : rank_sentence(tokenized_sentences[i]), reverse=True)
    for i in range(top_k):
        print('-' * 50)
        print(sentences[idxs[i]])
        print(len(tokenized_sentences[idxs[i]]), rank_sentence(tokenized_sentences[idxs[i]]))
        print([x for x in tokenized_sentences[idxs[i]] if x in medcomp.medical_terms])
        print([x for x in tokenized_sentences[idxs[i]] if x not in medcomp.medical_terms])
        print(tokenizer.ids2string(tokenizer.string2ids(sentences[idxs[i]])))

In [4166]:
sample_sentences()

--------------------------------------------------
exam is otherwise remarkable for a large mass-like area of opacification above the right hilum with adjacent surgical sutures, as well as right upper lobe volume loss and asymmetrical right apical thickening.
36 20
['remarkable', 'large', 'mass', 'area', 'opacification', 'above', 'right', 'hilum', 'adjacent', 'surgical', 'sutures', 'right', 'upper', 'lobe', 'volume', 'loss', 'asymmetrical', 'right', 'apical', 'thickening']
['exam', 'is', 'otherwise', 'for', 'a', '-', 'like', 'of', 'the', 'with', ',', 'as', 'well', 'as', 'and', '.']
<s> exam is otherwise remarkable for a large mass like area of opacification above the right hilum with adjacent surgical sutures , as well as right upper lobe volume loss and asymmetrical right apical thickening . </s>
--------------------------------------------------
the final radiograph in the series shows repositioning of the right pic line from a right internal jugular vein to the estimated location of

In [4476]:
def evaluate_pair(i = None, j = None):
    word = random.choice(tokenizer.id2token)
    _i, _j = random.choices(word2idxs[word], k=2)
    if i is None: i = _i
    if j is None: j = _j
    si = sentences[i]
    sj = sentences[j]
    print('***', i, si)
    print('----')
    print('***', j, sj)
    score_debug = medcomp.score__debug(
        tokenizer.string2ids(si),
        tokenizer.string2ids(sj),
        tokenizer, verbose=False)
    score = medcomp.score(
        tokenizer.string2ids(si),
        tokenizer.string2ids(sj))
    assert score_debug == score
    print('score_debug =', score_debug, 'score =', score)

In [4490]:
evaluate_pair()

*** 889075 improving subcutaneous and intramuscular air.
----
*** 889070 there is also improvement of the subcutaneous and intramuscular gas seen on the right side of the body.
------------------
ground truth
------------------
1-grams (3): subcutaneous, intramuscular, air
ignored (5): <s>, improving, and, ., </s>
------------------
generated
------------------
1-grams (7): amelioration, subcutaneous, intramuscular, gas, right, side, bodies
ignored (14): <s>, there, is, also, of, the, and, seen, on, the, of, the, ., </s>
inter_size = [2, 1, 0]
score_debug = 0.21666666666666667 score = 0.21666666666666667


In [152]:
# with open('./medical_terms__aux.txt', 'a') as f:
#     for w in tokenizer.id2token:
#         f.write(f'{w}\n')

In [4397]:
def compute_weights():
    
    # load medical terms
    with open(MEDICAL_TERMS_PATH) as f:
        medical_terms = [line.strip() for line in f.readlines()]
    
    # load medical synonyms
    with open(MEDICAL_SYNONYMS_PATH) as f:
        medical_synonyms = [line.strip().split() for line in f.readlines()]
    term2synonym = { x:x for x in medical_terms }
    for row in medical_synonyms:
        term0 = row[0]
        for i in range(1, len(row)):
            term2synonym[row[i]] = term0
    
    # compute frequencies
    freqs = [dict() for _ in range(4)]
    counts = [0] * 4
    for sentence in tokenized_sentences:
        medical_sentence = []
        for token in sentence:
            synonym = term2synonym.get(token, None)
            if synonym is not None:
                medical_sentence.append(synonym)
        for k in range(min(4, len(medical_sentence))):
            f_k = freqs[k]
            for i in range(len(medical_sentence) - k):
                if k == 0:
                    key = medical_sentence[i]
                else:
                    key = tuple(medical_sentence[i:i+k+1])
                f_k[key] = f_k.get(key, 0) + 1
                counts[k] += 1
    
    # compute weights
    weights = [
        { key : math.log(counts[k] / freqs[k][key]) for key in freqs[k].keys() }
        for k in range(4)
    ]
    return freqs, weights

In [4398]:
_freqs, _weights = compute_weights()

In [4399]:
from medvqa.utils.files import save_to_pickle
from medvqa.utils.common import CACHE_DIR
import os

In [4400]:
save_to_pickle(_weights, os.path.join(CACHE_DIR, 'medical_terms_weights.pkl'))

In [4422]:
weightmedcomp = WeightedMedicalCompleteness(tokenizer, device='cpu')

In [4423]:
def evaluate_pair__weighted(i = None, j = None):
    word = random.choice(tokenizer.id2token)
    _i, _j = random.choices(word2idxs[word], k=2)
    if i is None: i = _i
    if j is None: j = _j
    si = sentences[i]
    sj = sentences[j]
    print('***', i, si)
    print('----')
    print('***', j, sj)
    score = weightmedcomp.score(
        tokenizer.string2ids(si),
        tokenizer.string2ids(sj))
    print('score =', score)

In [4449]:
# evaluate_pair__weighted(1023895, 376450)
evaluate_pair__weighted()

*** 120631 post-surgical changes in the right lower lobe are again seen with chain sutures abutting the oblique fissure.
----
*** 245980 chain sutures within the left upper and lower lung fields are compatible with prior wedge resections.
[71.08388006555843, 94.5753593236537, 100.64032808444145, 97.54173147961983]
[59.7469520233614, 72.22699991291432, 79.0491533630988, 74.94881956078274]
[22.037562117814087, 9.19059404597967, 0, 0]
score = 0.11177094542986725


In [4450]:
evaluate_pair(120631 , 245980 )

*** 120631 post-surgical changes in the right lower lobe are again seen with chain sutures abutting the oblique fissure.
----
*** 245980 chain sutures within the left upper and lower lung fields are compatible with prior wedge resections.
------------------
ground truth
-----
1-grams (10): post, surgeries, right, low, lobe, chain, suture, abut, oblique, fissure
ignored (11): <s>, changes, in, the, are, again, seen, with, the, ., </s>
------------------
generated
-----
1-grams (9): chain, suture, left, theupper, low, lung, field, wedge, resected
ignored (10): <s>, within, the, and, are, compatible, with, prior, ., </s>
intersec_size = [3, 1, 0, 0]
score = 0.10835913312693499
